# 14. Hybrid Inference Strategy 4: Structured Reasoning + Verifier-Guided Scoring

**Objective**  
Balance rule-based logic and verifier trust by adding structured dependency mapping and weighted scoring heuristics atop the v3 pipeline.

1. **Question Parsing (QP)**  
   - Deterministic LLaMA-3 beam-search parse.  
   - No verifier filtering.

2. **CoT Parsing (CP)**  
   - Generate 5 candidates (2 beam + 3 sampled).  
   - Parse into `{statement, evidence, Verification}`.

3. **Advanced Post-Processing**  
   - **Enhanced Evidence Extraction** via condition references & entity overlap.  
   - **Strict Verification Normalization** to `"true"`/`"false"` patterns.  
   - **Reasoning Chains**: add `reasoning_step` and `related_to`.

4. **Candidate Scoring**  
   - Compute a **weighted score** combining:
     - Step count & verification balance  
     - Structure (intra-step connections)  
     - Evidence length/clarity  
     - CP verifier logits (scaled)

5. **Output**  
   Same JSON schema as before.

---

## Evaluation

| Metric                         | v4 Score |
|--------------------------------|----------|
| **Question_Macro_F1**          | 0.7658   |
| **Statement_Macro_F1**         | 0.4185   |
| **Statement_Evidence_Macro_F1**| 0.1214   |
| **Reasoning_F1**               | 0.0939   |

> _v4 achieves the highest Statement_F1 yet but with lower evidence/reasoning metrics, reflecting its stricter structural filtering and heuristic scoring._



## Setup and Thresholds

In [1]:
# Install core evaluation utilities
!pip install -q evaluate
!pip install json5

!pip uninstall -y nltk
!pip install -q --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.5 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download("punkt_tab")
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import unsloth  # Must come first for 4-bit LoRA
import torch, gc, json, re, ast, html, numpy as np
from torch.nn.functional import log_softmax
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    pipeline
)
from datasets import load_dataset
from collections import Counter
import math


# Paths & thresholds
INPUT       = "/content/drive/MyDrive/llm-sr-project/testingData-blank.json"
OUTPUT      = "/content/drive/MyDrive/llm-sr-project/results_hybrid_approach5.json"
QP_LM_PATH  = "/content/drive/MyDrive/llm-sr-project/finetuned_llama3_question_parsing"
CP_LM_PATH  = "/content/drive/MyDrive/llm-sr-project/finetuned_llama3_cot_parsing"
QP_VER_PATH = "/content/drive/MyDrive/deberta-qparse-verifier"
CP_VER_PATH = "/content/drive/MyDrive/deberta-cotparse-verifier"


THR_QP = 0.70
THR_CP = 0.58
device = "cuda" if torch.cuda.is_available() else "cpu"

## Prompt Templates

In [ ]:
# In-Context Learning (ICL) Prompts

QP_DEMON = '''The question is:

There are 6 volunteers: A, B, C, D, E and F. They will be assigned to either Project Alpha or Project Beta. Each person works on exactly one project. This assignment must satisfy:
(1) If A works on Alpha, then B works on Beta.
(2) If C works on Alpha, then D and E work on Beta.
(3) F works on a different project than E.
(4) D must work on a different project than A.
(5) If F works on Alpha, then B works on Alpha.

If A works on Beta, which of the following must be true?
A. B works on Alpha
B. C works on Beta
C. D works on Alpha
D. F works on Beta

The parsing result is:

[
  "There are 6 volunteers: A, B, C, D, E and F. They will be assigned to either Project Alpha or Project Beta. Each person works on exactly one project.",
  "If A works on Alpha, then B works on Beta",
  "If C works on Alpha, then D and E work on Beta",
  "F works on a different project than E",
  "D must work on a different project than A",
  "If F works on Alpha, then B works on Alpha",
  "A works on Beta"
]
'''

QP_TEMPLATE = '''Given a question, extract all relevant information from the question that would help to solve it.

This includes:
- General setup information (e.g., number of people, projects involved)
- Explicit facts given in the question
- All logical constraints or conditions

Output only a JSON list and nothing else. Follow the format shown in the example.

Example:

{demon}

Now, the question is:

{question}

Your output:
'''

CP_DEMON = '''The question is:

There are 6 volunteers: A, B, C, D, E and F. Each person works on exactly one project.

Conditions:
(1) If A works on Alpha, then B works on Beta.
(2) If C works on Alpha, then D and E work on Beta.
(3) F works on a different project than E.
(4) D must work on a different project than A.
(5) If F works on Alpha, then B works on Alpha.

Question:
If A works on Beta, which of the following must be true?

CoT:
Since A works on Beta, Condition (1) is not triggered. Condition (2) is not triggered since C's assignment is unknown. Condition (3) doesn't give anything because E's assignment is unspecified. Condition (4) says D must work on a different project than A, so D must work on Alpha. Condition (5) depends on F, which is unknown.

Parsing result:

[
  {
    "statement": "Condition (1) is not applicable",
    "evidence": "Condition (1): If A works on Alpha, then B works on Beta. | A is working on Beta",
    "Verification": "false"
  },
  {
    "statement": "Condition (2) is not applicable",
    "evidence": "Condition (2): If C works on Alpha, then D and E work on Beta. | C's assignment is unknown",
    "Verification": "false"
  },
  {
    "statement": "Condition (3) does not provide any info",
    "evidence": "Condition (3): F works on a different project than E. | E's assignment is unknown",
    "Verification": "false"
  },
  {
    "statement": "D must work on Alpha",
    "evidence": "Condition (4): D must work on a different project than A, and A is working on Beta",
    "Verification": "true"
  },
  {
    "statement": "Condition (5) is not applicable",
    "evidence": "Condition (5): If F works on Alpha, then B works on Alpha. | F's assignment is unknown",
    "Verification": "false"
  }
]
'''

CP_TEMPLATE = '''You are a reasoning assistant. Based on the question, conditions, and chain-of-thought (CoT), extract every inference or non-inference step as a JSON object.

For each CoT sentence that either:
  1. Refers to a condition (e.g. "Condition (2) …")
  2. Starts with an inference cue ("Since", "Therefore", "This means", "We can deduce", etc.)

Produce one object with:
  • "statement": the new claim you read in that CoT sentence (don't quote the entire sentence—just the core inference).
  • "evidence":
      – if the claim restates a constraint, use the exact line from the **Conditions** block,
      – otherwise, use the CoT fragment that you extracted it from.
  • "Verification":
      – MUST BE EXACTLY `"false"` if the sentence rejects or blocks a condition (contains "not applicable", "does not provide", etc.),
      – MUST BE EXACTLY `"true"` in all other cases.
  • "reasoning_step": number indicating the logical step in the reasoning chain (starting from 1)
  • "related_to": [list of numbers] indicating which previous steps this one builds upon (optional)

Keep the objects in the same order as they appear in the CoT.

IMPORTANT: "Verification" field MUST ONLY contain the string "true" or "false" (lowercase) and nothing else.

Example:

{demon}

Now, given:

Question:
{question}

Conditions:
{conditions}

Chain-of-Thought:
{cot}

Your output:
'''

## Helper Functions

In [ ]:
def clean_quotes(t):
    return (t.replace('"','"').replace('"','"').replace("'","'").replace("'","'"))

def normalize_text(t):
    t = clean_quotes(t)
    t = re.sub(r'\?\s(?=[A-Z])', ', ', t)
    t = re.sub(r'(?<=[a-zA-Z])\.(?=[A-Z])', '. ', t)
    t = re.sub(r'(?<![A-Da-d])\\n(?!\s?[A-Da-d]\\.)', ' ', t)
    return html.unescape(t).strip()

def extract_json(raw):
    raw = raw.strip()
    i = raw.find('[')
    if i < 0: return []
    depth = 0
    for j,ch in enumerate(raw[i:], i):
        if ch=='[': depth+=1
        elif ch==']': depth-=1
        if depth==0:
            blk = raw[i:j+1]
            for p in (json.loads, ast.literal_eval):
                try: return p(blk)
                except: pass
            return []
    return []

def score_verifier_batch(prem_list, hyp_list, tok, mod):
    enc = tok(prem_list, hyp_list, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        logits = mod(**enc).logits
    return torch.softmax(logits, dim=1)[:, 1].tolist()

def clean_qp(qp_list):
    return [s for s in qp_list if not re.match(r'^[A-Da-d][\.:\)]', s.strip()) and "Option" not in s and "following" not in s]

# Optimized evidence enhancement function - focused on condition matching and CoT coherence
def enhance_evidence(statement, evidence, cot, conditions):
    # If evidence is already meaningful, preserve it
    if evidence and len(evidence.strip()) >= 20:
        return evidence

    # Check for condition references
    condition_match = re.search(r'(?:condition|constraint)\s*\(?(\d+)\)?', statement.lower())
    if condition_match:
        condition_num = int(condition_match.group(1))
        if 1 <= condition_num <= len(conditions):
            # Check for not applicable cases
            if any(x in statement.lower() for x in ["not applicable", "not triggered", "doesn't apply", "no information"]):
                return f"{conditions[condition_num-1]} | This is not applicable because the condition's premise is not satisfied."
            return conditions[condition_num-1]

    # For reasoning steps, extract key entities and find relevant CoT sentences
    statement_entities = set(re.findall(r'\b[A-F]\b', statement))
    if statement_entities:
        sentences = re.split(r'[.!?]', cot)
        for sent in sentences:
            sent = sent.strip()
            if len(sent) < 10:
                continue

            sent_entities = set(re.findall(r'\b[A-F]\b', sent))
            if sent_entities and len(statement_entities.intersection(sent_entities)) >= 1:
                if "must" in statement.lower() and "must" in sent.lower():
                    # High priority evidence for inference steps
                    return sent + "."
                if len(statement_entities.intersection(sent_entities)) >= 2:
                    # Good evidence with multiple entity matches
                    return sent + "."

    # Extract key terms from statement for a fallback match
    statement_kw = set(re.findall(r'\b[a-zA-Z]{4,}\b', statement.lower()))
    sentences = re.split(r'[.!?]', cot)

    best_score = 0
    best_sent = ""
    for sent in sentences:
        sent = sent.strip()
        if len(sent) < 10:
            continue

        sent_kw = set(re.findall(r'\b[a-zA-Z]{4,}\b', sent.lower()))
        overlap = len(statement_kw.intersection(sent_kw))

        if overlap > best_score:
            best_score = overlap
            best_sent = sent

    if best_score >= 2:
        return best_sent + "."

    # Keep original evidence if nothing better found
    return evidence or "Based on logical analysis."

# Improved verification function to ensure proper values without over-filtering
def normalize_verification(statement, verification):
    # If verification field is missing or invalid, infer from statement content
    if verification not in ["true", "false"]:
        # Detect patterns suggesting inapplicability
        if any(x in statement.lower() for x in [
                "not applicable", "doesn't apply", "does not apply",
                "not triggered", "no information", "doesn't provide",
                "doesn't give", "cannot determine"
            ]):
            return "false"
        return "true"  # Default to true if no negative indicators

    return verification

# Enhanced function for building reasoning connections
def build_reasoning_connections(steps):
    # First pass - assign step numbers
    for i, step in enumerate(steps):
        step["reasoning_step"] = i + 1

    # Second pass - build connections based on entities and topics
    for i, curr_step in enumerate(steps):
        if i == 0:
            continue  # Skip first step

        # Extract entities from current step
        curr_entities = set(re.findall(r'\b[A-F]\b', curr_step["statement"]))

        # Only add dependencies for inference steps (typically marked "true")
        if curr_step.get("Verification") == "true":
            related_steps = []

            # Find related previous steps
            for j, prev_step in enumerate(steps[:i]):
                # Extract entities from previous step
                prev_entities = set(re.findall(r'\b[A-F]\b', prev_step["statement"]))

                # Check for entity overlap - strong indicator of dependency
                if prev_entities and curr_entities and prev_entities.intersection(curr_entities):
                    related_steps.append(j + 1)

                # Check for condition reference in inference
                if prev_step.get("Verification") == "false" and "condition" in prev_step["statement"].lower():
                    condition_match = re.search(r'condition\s*\(?(\d+)\)?', prev_step["statement"].lower())
                    if condition_match:
                        # Potential dependency on condition state
                        related_steps.append(j + 1)

            # Add unique dependencies
            if related_steps:
                curr_step["related_to"] = sorted(set(related_steps))

    return steps

# Custom weighted selection that balances verifier with evidence and reasoning quality
def compute_candidate_score(candidate, verifier_scores=None):
    if not candidate:
        return -float('inf')  # Heavily penalize empty candidates

    # Calculate quality metrics
    metrics = {
        "num_steps": len(candidate),
        "true_steps": sum(1 for s in candidate if s.get("Verification") == "true"),
        "has_reasoning": any("reasoning_step" in s for s in candidate),
        "has_connections": sum(1 for s in candidate if "related_to" in s),
        "avg_evidence_len": np.mean([len(s.get("evidence", "")) for s in candidate]),
    }

    # Combine metrics with carefully tuned weights
    base_score = (
        metrics["num_steps"] * 0.2 +  # More steps is good
        metrics["true_steps"] * 0.3 +  # True verifications are important
        (metrics["true_steps"] > 0) * 1.0 +  # Must have at least one true step
        (metrics["has_connections"] / max(1, metrics["num_steps"])) * 1.5 +  # More connections is good
        min(1.0, metrics["avg_evidence_len"] / 50.0) * 0.5  # Better evidence is good (up to a point)
    )

    # Incorporate verifier influence with reduced weight
    if verifier_scores and len(verifier_scores) > 0:
        # Use mean rather than sum to keep scale consistent regardless of candidate length
        verifier_score = np.mean([score for score in verifier_scores])
        combined_score = (0.65 * base_score) + (0.35 * verifier_score * 5.0)
        return combined_score

    return base_score

## Load Models and Verifiers

In [ ]:
# QP LM - Using beam search
qp_tok = AutoTokenizer.from_pretrained(QP_LM_PATH)
qp_tok.model_max_length = 1024
qp_mod = AutoModelForCausalLM.from_pretrained(QP_LM_PATH).to(device)
qp_pipe = pipeline("text-generation", model=qp_mod, tokenizer=qp_tok,
                   return_full_text=False, do_sample=False,
                   num_beams=5, early_stopping=True,       =
                   max_new_tokens=512, batch_size=4)

# CP LM - Using beam search
cp_tok = AutoTokenizer.from_pretrained(CP_LM_PATH)
cp_tok.model_max_length = 2048
cp_mod = AutoModelForCausalLM.from_pretrained(CP_LM_PATH).to(device)

# Generate 2 beam candidates + 3 sampled candidates = 5 total
cp_pipe = pipeline(
     "text-generation",
     model=cp_mod, tokenizer=cp_tok,
     return_full_text=False,
     do_sample=False, num_beams=5, num_return_sequences=2,
     max_new_tokens=1024,
     batch_size=4
)

cp_sampler = pipeline(
    "text-generation",
    model=cp_mod, tokenizer=cp_tok,
    return_full_text=False,
     do_sample=True, temperature=0.8, num_return_sequences=3,
     max_new_tokens=1024,
     batch_size=4
)

# Load Verifiers
def load_verifiers():
    global qv_tok, qv_mod, cv_tok, cv_mod
    qv_tok = AutoTokenizer.from_pretrained(QP_VER_PATH)
    qv_mod = AutoModelForSequenceClassification.from_pretrained(QP_VER_PATH).to(device)
    cv_tok = AutoTokenizer.from_pretrained(CP_VER_PATH)
    cv_mod = AutoModelForSequenceClassification.from_pretrained(CP_VER_PATH).to(device)
    return qv_tok, qv_mod, cv_tok, cv_mod


qv_tok, qv_mod, cv_tok, cv_mod = load_verifiers()

## Hybrid Inference Function

In [ ]:
# Hybrid inference approach v4
def process_one(example):
    q_raw, cot_raw = example["question"], example["cot"]
    sel_idx, ans = example.get("sel_idx"), example.get("answer")
    q, cot = normalize_text(q_raw), normalize_text(cot_raw)

    # Generate QP output
    prompt = QP_TEMPLATE.format(demon=QP_DEMON, question=q)
    raw_qp = qp_pipe(prompt, max_new_tokens=512)
    if not isinstance(raw_qp, list):
        raw_qp = [raw_qp]
    best_qp = clean_qp(extract_json(raw_qp[0]["generated_text"]))

    # Generate CP parses with standard approach
    conds_str = json.dumps(best_qp, ensure_ascii=False)
    prompt_cp = CP_TEMPLATE.format(
        demon      = CP_DEMON,
        question   = q,
        conditions = conds_str,
        cot        = cot
    )

    # Get 5 candidates (2 beam + 3 sampled)
    raw_beams = cp_pipe(prompt_cp, max_new_tokens=1024)
    raw_samples = cp_sampler(prompt_cp, max_new_tokens=1024)
    raw_cp_outs = (raw_beams if isinstance(raw_beams, list) else [raw_beams]) \
            + (raw_samples if isinstance(raw_samples, list) else [raw_samples])

    # Flatten outputs
    raw_cp_flat = [
        item
        for sub in raw_cp_outs
        for item in (sub if isinstance(sub, list) else [sub])
    ]

    # Parse candidates with improved processing
    cps_candidates = []
    for out in raw_cp_flat:
        parsed = extract_json(out["generated_text"])
        if not parsed:
            continue

        seen = set()
        cleaned = []

        for st in parsed:
            # Handle unexpected objects
            if not isinstance(st, dict):
                continue

            stmt = st.get("statement","").strip()
            ev   = st.get("evidence","").strip() or "logical deduction"

            # Skip very short or duplicate statements
            if len(stmt) < 3 or (stmt, ev) in seen:
                continue

            # Enhance evidence quality using context information
            ev = enhance_evidence(stmt, ev, cot, best_qp)

            # Normalize verification value
            ver = normalize_verification(stmt, st.get("Verification","true"))

            # Store the step
            seen.add((stmt, ev))
            cleaned.append({"statement": stmt, "evidence": ev, "Verification": ver})

            # Preserve any existing reasoning metadata
            if "reasoning_step" in st:
                cleaned[-1]["reasoning_step"] = st["reasoning_step"]

            if "related_to" in st:
                cleaned[-1]["related_to"] = st["related_to"]

        if cleaned:
            # Only apply if the candidate doesn't already have reasoning structure
            if not any("related_to" in step for step in cleaned):
                cleaned = build_reasoning_connections(cleaned)

            cps_candidates.append(cleaned)

    # Fallback for empty results
    if not cps_candidates:
        cps_candidates = [[]]

    # Score candidates with verifier + quality metrics
    premise = f"Question:\n{q}\n\nConditions:\n" + "\n".join(f"- {s}" for s in best_qp) + f"\n\nCoT:\n{cot}"

    # Get verifier scores for each candidate
    verifier_scores_by_candidate = []
    for cp_list in cps_candidates:
        if not cp_list:
            verifier_scores_by_candidate.append([])
            continue

        prems = [premise] * len(cp_list)
        hyps = [f"Statement: {st['statement']}\nEvidence: {st['evidence']}" for st in cp_list]
        scores = score_verifier_batch(prems, hyps, cv_tok, cv_mod)
        verifier_scores_by_candidate.append(scores)

    # Compute weighted scores that balance verifier with quality metrics
    combined_scores = []
    for i, candidate in enumerate(cps_candidates):
        verifier_scores = verifier_scores_by_candidate[i] if i < len(verifier_scores_by_candidate) else []
        score = compute_candidate_score(candidate, verifier_scores)
        combined_scores.append(score)

    # Select best candidate
    best_idx = int(np.argmax(combined_scores)) if combined_scores else 0
    best_cp = cps_candidates[best_idx] if cps_candidates and best_idx < len(cps_candidates) else []


    for i, step in enumerate(best_cp):
        step["reasoning_step"] = i + 1

    return {
        "question": q_raw,
        "question_parsing": best_qp,
        "answer": ans,
        "id": example["id"],
        "cot": cot_raw,
        "cot_parsing": best_cp,
        "sel_idx": sel_idx
    }

## Batch and Run

In [ ]:
def process_batch(batch):
    outs = [process_one({
        "question": batch["question"][i],
        "cot":      batch["cot"][i],
        "id":       batch["id"][i],
        "sel_idx":  batch.get("sel_idx", [None]*len(batch["id"]))[i],
        "answer":   batch.get("answer", [None]*len(batch["id"]))[i],
    }) for i in range(len(batch["question"]))]

    return {
        "question":        [o["question"]        for o in outs],
        "question_parsing":[o["question_parsing"]for o in outs],
        "answer":          [o["answer"]          for o in outs],
        "id":              [o["id"]              for o in outs],
        "cot":             [o["cot"]             for o in outs],
        "cot_parsing":     [o["cot_parsing"]     for o in outs],
        "sel_idx":         [o["sel_idx"]         for o in outs],
    }

if __name__=="__main__":
    gc.collect()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    try:
        ds = load_dataset("json", data_files={"test": INPUT})["test"]
        print(f"Loaded dataset with {len(ds)} examples")

        # Add more descriptive logging
        print("Starting batch processing with optimally balanced verifier...")
        print(f"Target metrics to exceed: QP_F1=0.7526, ST_F1=0.4015, ST_EV_F1=0.1849, RSN_F1=0.1405")

        out_ds = ds.map(
            process_batch,
            batched=True,
            batch_size=2,
            remove_columns=ds.column_names
        )

        print(f"Processing complete. Writing results to {OUTPUT}")
        out_ds.to_json(OUTPUT, orient="records", lines=False)
        print("✅ Done — saved to", OUTPUT)
    except Exception as e:
        print(f"Error occurred: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()

## Transform Predictions

In [ ]:
import json

INPUT_PATH  = "/content/drive/MyDrive/llm-sr-project/results_hybrid_approach5.json"
OUTPUT_PATH ="/content/drive/MyDrive/llm-sr-project/final_results_hybrid_approach5.json"

def transform_example(ex):
    # reorder each cot_parsing entry: statement → evidence → Verification
    reordered = []
    for step in ex.get("cot_parsing", []):
        reordered.append({
            "statement":    step.get("statement"),
            "evidence":     step.get("evidence"),
            "Verification": step.get("Verification"),
        })

    return {
        "question":         ex.get("question"),
        "question_parsing": ex.get("question_parsing"),
        "answer":           ex.get("answer"),
        "id":               ex.get("id"),
        "cot":              ex.get("cot"),
        "cot_parsing":      reordered,
        "sel_idx":          ex.get("sel_idx"),
    }

def main():
    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        examples = json.load(f)

    structured = [transform_example(ex) for ex in examples]

    with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
        json.dump(structured, f, ensure_ascii=False, indent=2)

    print(f"Wrote {len(structured)} examples to {OUTPUT_PATH}")

if __name__ == "__main__":
    main()

## Evaluate

In [3]:
EVAL_SCRIPT = "/content/drive/MyDrive/llm-sr-project/eval.py"
PREDICTION_PATH = "/content/drive/MyDrive/llm-sr-project/results_hybrid_approach5.json"
REFERENCE_PATH = "/content/drive/MyDrive/llm-sr-project/test-reference.json"

!python {EVAL_SCRIPT} \
  --prediction {PREDICTION_PATH} \
  --reference {REFERENCE_PATH} \
  --question_threshold 0.95 \
  --statement_threshold 0.9 \
  --relation_threshold 0.9

2025-05-17 18:02:50.039021: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-17 18:02:50.058879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747504970.080446    5563 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747504970.086994    5563 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-17 18:02:50.108198: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr